# Part II C3 Petrology - Practical 20: MORB Global Systematics
*Simon Matthews (sm905@cam.ac.uk), Lent 2023*

In this practical you will extract and analyse geochemical and geophysical data for the global mid-ocean ridge system with the aim of identifying what the primary controls on the composition of MORB are. The practical will also demonstrate how we can use computers and relatively simple python scripts to handle large datasets efficiently and perform otherwise time consuming calculations.

All the practicals in this part of the course are contained in Jupyter Notebooks which can be run in the cloud on a service called myBinder. This means they should work on *any* computer, provided it has a modern web browser. The notebooks allow you to run code alongside text and images and allow you to conveniently save the output from the code. Whole scientific papers (e.g. https://mybinder.org/v2/gh/kaylai/vesical-binder/HEAD?filepath=Manuscript.ipynb) can be written in Jupyter Notebooks! **However, all the work you do is stored on a server in the cloud and will disappear when you shut the window, so make sure you download a copy!** To do this, click `File` in the top-left corner, and find `Download` on the menu that appears.

A summary of the practical will be available online after the end of the practical class, but don't rely on this, use the demonstrators! Some of the tasks are deliberately designed to require discussions with other students and the demonstrators.

## 1. Retrieving and plotting bathymetry data using pyGMT
There are multiple ways of interacting with these datasets but we will use a package called pyGMT, which is a python interface to the GMT ([Generic Mapping Tools](https://www.generic-mapping-tools.org/)) software. GMT is widely used in Earth and Marine Sciences. You can read more about the bathymetry data [here](https://www.gmrt.org/).

First we must import the `pygmt` package so the notebook has access to its code and mapping know-how:

In [ ]:
import pygmt

Now we will extract some of the data GMT is packaged with and assign it to variables so that we can use it more conveniently.

The first dataset is a grid of altitude/bathymetry covering the whole planet

In [ ]:
grid = pygmt.datasets.load_earth_relief()
grid

The second dataset records the location of spreading ridges:

In [ ]:
points = pygmt.datasets.load_sample_data(name='ocean_ridge_points')
points

Now we can combine both of these datasets to extract the bathymetry along the ridge axes:

In [ ]:
track = pygmt.grdtrack(points=points, grid=grid, newcolname="bathymetry") 
track

Let's see what this looks like on a map:

In [ ]:
# Create a GMT figure and store it in variable fig:
fig = pygmt.Figure()

# Plot the earth relief grid on Cylindrical Stereographic projection
fig.basemap(region="g", frame=True, projection="Cyl_stere/0/45/15c") # this sets the projection and the region and the size of the plot (15cm)
fig.grdimage(grid=grid, cmap="gray") # the plots up the ocean bathymetry with a boring gray colour scale
fig.coast(land="#666666") # Mask land areas

# Generate a color map for the bathymetry
pygmt.makecpt(cmap='terra', series=[-5000, 5000])

# Plot using circles (c) of 0.15 cm, the sampled bathymetry points
# Points are colored using elevation values (normalized for visual purposes)
fig.plot(
    x=track.longitude,
    y=track.latitude,
    style="c0.15c",
    cmap=True, # Use the colormap generated above
    fill=track.bathymetry, # Colour according to bathymetry
)
fig.colorbar(frame=["a2500", "x+lElevation", "y+lm"])
fig.show()

You should visit the [pygmt](https://www.pygmt.org/latest/) manual pages and look at the information about [projections](https://www.pygmt.org/latest/projections/index.html) and [setting the region](https://www.pygmt.org/latest/tutorials/regions.html) to get a quick feel for what the `projection` and `region` arguments mean. Please ask if this does not make sense. 

Most of the plots we make in these practicals (and probably when you use python to make plots in general) uses a library called **matplotlib**. Unfortunately, pyGMT works quite differently to matplotlib, and (in my opinion) is far less intuitive than matplotlib. It does produce much higher quality maps and provides us with convenient access to bathymetry datasets, so it is worth the extra hassle!

## 2. Extract and plot chemical data for oceanic basalts from PETDB

In the practical folder is a spreadsheet containing a large compilation of chemical data collected from rocks at present-day spreading ridges. The spreadsheet was downloaded from the [PETDB](https://www.earthchem.org/petdb) website. Try running a search for some part of the Earth, and have a look at the data that is returned.

### Q2.1: What categories of rocks in the database are likely to be the most and least useful for understanding magmatic processes? Ask a demonstrator if you're not sure where to find this information.

*Your answer here...*

The **pandas** python package provides a toolkit for importing and manipulating large datasets:

In [ ]:
import pandas as pd

Adding `as pd` after the import command means that we can access all the functions using the name `pd`, saving us from typing out `pandas` each time!

We can use pandas to read in an Excel file (telling it to skip the first 5 rows which hold a title and some information from PETDB):

In [ ]:
petdb = pd.read_excel("spreading_ridge_rocks.xlsx", header=5)

It's always a good idea to check the data import has worked correctly, so we can look at the very top of the table:

In [ ]:
petdb.head()

While this gives us a convenient view, it skips some of the columns. To see all of the columns that exist in the table:

In [ ]:
petdb.columns

Importantly for us, we have the latitude, longitude, and the oxide composition (in wt%).

### Q2.2: How have the columns FE2O3, FE2O3T, FEO, and FEOT been populated in the spreadsheet, and what is their meaning?

*Your answer here...*

To plot the data in this part of the practical we will use matplotlib, which is conventially imported like this:

In [ ]:
import matplotlib.pyplot as plt

We can plot the whole dataset at once, which will allow us to check data quality as well as get an overview of it. Here is an example of one plot using matplotlib, with a little bit of customisation:

In [ ]:
fig, ax = plt.subplots(dpi=100) # Make a figure, and put a set of axes on it, set the display resolution as 350 DPI.

# Add some scatter points to the axis
ax.scatter(petdb.MGO, petdb.NA2O)

# Set the axis labels (notice how you make a subscript appear):
ax.set_xlabel('MgO (wt%)')
ax.set_ylabel('Na$_2$O (wt%)')

# Display the plot
plt.show()

The things you can do to customise how matplotlib displays a plot are virtually endless. Have a look at the [matplotlib examples gallery](https://matplotlib.org/stable/gallery/index) to get an idea for this.

### Q2.3: Do you notice any issues with the data?

*You might want to change the plot settings to zoom into parts of the dataset. You can do this with the lines `ax.set_xlim(..., ...)` and `ax.set_ylim(..., ...)`.*

*Your answer here...*

We should bear these issues in mind when we start using the dataset. They might be things we can ignore, or we might want to filter the data before we use it.

### Q2.4: Is there any pattern you notice in the global dataset when MgO is plotted against Na$_2$O?

*Your answer here...*

### Q2.5: Produce versions of the plot coloured for longitude and latitude

*To add a colour scale to the points you can use the argument `c=...` where you enter the sequence of data in place of "...". Try working out how to do this from the code above, and ask a demonstrator if you get stuck.*

In [ ]:
# YOUR CODE HERE


Though its tricky to pick out all the data on these plots, you should notice that there is some systematic variation between where the data sits in Na$_2$O vs MgO and its location.

### Q2.6: Perhaps a better way of viewing it is to colour code the symbols on a map. Repurpose the code that produced the map above to make such a map.

*The colormap used above is great for plotting topography and bathymetry, but not so great for other variables. Find a better one by using the [GMT colour scale reference](https://docs.generic-mapping-tools.org/latest/cookbook/cpts.html#built-in-color-palette-tables-cpt)*

In [ ]:
# YOUR CODE HERE

### Q2.7 Is there any coherence between Na<sub>2</sub>O content and bathymetry? Any noteable extremes? Why?

*Your answer here...*

## 3. Controls on the Na$_2$O concentration in MORB

You should have found some systematic pattern between Na<sub>2</sub>O and bathymetry. If you didn't, speak to a demonstrator. This section will explore this in a bit more detail and in a more rigorous fashion.

These relationships between the major element composition of MORB and geophysical indicators of the extent of melting (such as crustal thickness or depth of seafloor at the spreading ridge axis) were used by Charlie Langmuir and co-workers to argue for the presence of a global array in MORB composition that is controlled by mantle properties (e.g. [Klein & Langmuir, JGR, 1987](https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1029/JB092iB08p08089)).

In order to isolate chemical variations associated with mantle, rather than crustal, processes these authors devised a chemical property known as Na<sub>8</sub>. Take a look at their Figure 1a and the surrounding text to see how this property is defined and used. 

### Q3.1: Briefly describe why comparison of Na<sub>8</sub> between different parts of the spreading ridge system should provide a clearer picture of mantle melting variations than the use of Na<sub>2</sub>O from above.

*Your answer here...*

### Q3.2: Why is it better to average over rift segments rather than using individual rock compositions and the bathymetry at the point they were collected?

*Your answer here...*

In the rest of the practical we will use the up-to-date datasets we imported above to produce a new version of the Na<sub>8</sub> -- bathymetry array. I will demonstrate how we might do this for one segment first...

To start with we will look at the Reykjanes Ridge, just to the south of Iceland. We will use a version of the plots above (combined with some data filtering) to extract representative values for the bathymetry and Na<sub>8</sub>. 

First, we can plot a map of the bathymetry for this segment. Notice we can use a higher resolution version of the bathymetry grid. To see the range of resolutions that GMT can call on, look at [the documentation page here](https://docs.generic-mapping-tools.org/6.0/datasets/earth_relief.html). We will also plot the locations of the rock samples in the database for the segment. To do this we must filter the database by location:

In [ ]:
## In PetDB, longitude goes from -180 to +180.  
petsam = petdb[(petdb.LONGITUDE > -26.5) & (petdb.LONGITUDE < -24.5) & (petdb.LATITUDE > 62.) & (petdb.LATITUDE < 63.)]

In [ ]:
## first of all, make another data grid using a subset of the global data. 
## The region defines the longitude and latitude bounds in the order WESN
## The resolution is 01m - which is 1 arcminute grid spacing. This should be sufficient 
grid = pygmt.datasets.load_earth_relief(resolution="01m", region=[-26.5, -24.5, 62, 63])



fig = pygmt.Figure()
fig.grdimage(grid=grid, projection="M15c", frame="a", cmap="haxby") # haxby is a colorscale that is OK for oceans
fig.grdcontour(grid=grid) # plot up contours
fig.colorbar(frame=["a1000", "x+lElevation", "y+lm"])
pygmt.makecpt(cmap="viridis", series=[1., 3.], background = "i") # color scale for Na2O - change series limits
# Plot using circles (c) of 0.5 cm, the sampled bathymetry points

fig.plot(
    x=petsam.LONGITUDE,
    y=petsam.LATITUDE,
    style="c0.5c",
    pen="white",
    cmap=True,
    fill=petsam.NA2O, # fill circle according color-scale
)


fig.colorbar(frame=["a1.0", "x+lNa2O", "y+lwt%"],position="JBC+o0c/4c")
fig.show()


So, this has generated a contoured and coloured bathymetry chart of the region of interest, along with the position of available basalt samples, these colour-coded by their Na<sub>2</sub>O contents. You can put the map in context by comparing to a global plot from GMT, the [GMRT website](https://www.gmrt.org/) or even Google Earth. 


Now we need to estimate Na<sub>8</sub> for the segment, which we could also by eye from a plot:

In [ ]:
f, a = plt.subplots()

a.scatter(petsam.MGO, petsam.NA2O, alpha=0.25, s=70)

a.set_xlabel('MgO (wt%)')
a.set_ylabel('Na$_2$O (wt%')

plt.show()

Notice on the plot above that each symbol is slightly transparent. I did this to see how dense the data is- there are many points overlapping with each other in a big cluster.

### Q3.3: Estimate (by eye) a value for Na<sub>8</sub> for this data.

*Your answer here...*

We could also do this a bit more rigorously by regressing the data to MgO = 8 wt%. A really simple way to do this is to run a linear regression over the whole dataset. Of course, someone has already written code that can do this in python, and made it available in a python package SciPy, which we can import here:

In [ ]:
from scipy.stats import linregress
import numpy as np

In this case I import a specific function from the library, rather than the whole library (as we did for pandas, for example). I also imported the library numpy, which provides lots of useful mathematical operations, and lets us define matrices etc.

We can look up how to use the linregress function with a handy Jupyter Lab trick:

In [ ]:
linregress?

Let's try running it:

In [ ]:
linregress(petsam.MGO, petsam.NA2O)

Damn! It didn't work. It turns out this is because some of the samples in the dataset have empty entried for some of the oxides. We could have filtered this out when we selected the data earlier, or we can do this when we call the function:

In [ ]:
linreg_result = linregress(petsam.MGO[(petsam.MGO>0)&(petsam.NA2O>0)], petsam.NA2O[(petsam.MGO>0)&(petsam.NA2O>0)])
linreg_result

We can see what this regression looks like by adding it to the plot below:

In [ ]:
f, a = plt.subplots()

a.scatter(petsam.MGO, petsam.NA2O, alpha=0.25, s=70)

x = np.array([5, 9])
y = linreg_result[0] * x + linreg_result[1]

a.plot(x, y, c='k')

a.set_xlabel('MgO (wt%)')
a.set_ylabel('Na$_2$O (wt%')

plt.show()

### Q3.4: How does this compare with the gradient of the lines used by Klein & Langmuir? Do you think this is a reliable way of estimating Na<sub>8</sub>? 

*Your answer here...*

The table below lists some spreading centres which will help us build a global axial depth - Na<sub>8</sub> array. How you populate this is **up to you**, but I suggest you work together as a class, each taking one segment (or work in pairs).

The Reykjanes peninsula has already been filled in with my favoured values, but maybe you disagree? If you disagree by much, speak to a demonstrator.


| Segment Name |	Longitude W |Longitude E | Latitude S | Latitude N | Crustal Thickness (km)| Axial Depth (m) | Na<sub>8</sub> |
|  :-:         | :-:            | :-:        | :-:        | :-:        | :-:                   | :-: | :-: |
| Reykjanes Example | -26.5 | -24.5 | 62.0 | 63.0 | 12 | 700 | 1.9 | 
| EPRR10 |	-104.4 |	-104.2 |	9.1 | 	10.1	| 6 | ? | ? |
| JUAN8 |	-129.5	| -128.9	| 46.54	| 47.5	| 7 | ? | ? |
| GALA6	| -97.8	| -96.5	| 2.1	| 2.2	| 5.5 |? | ? |
| GALA11 |	-92.1 |	-90.8 |	1.85 |	2.15 |	8 |? | ? |
| MARR23 |	-18.4 |	-17.6 |	67.9 |	68.9 |	9.5 |? | ? |
| MARR24 |	-18.8 |	-18.5 |	66.9 |	67.9 |	12 |? | ? |
| MARR86 |	-32.4 |	-32.2 |	37 |	37.5 |	6 |? | ? |
| SWIR67SWIR66 |	68 |	69.9 |	-26.4 |	-25.6 |	3.5 |? | ? |
| GAKK15 |	0 |	2.8 |	84.1 |	84.5 |	4 |? | ? |
| SEIR40 |	100.2 |	101.4 |	-47.7 |	-47.2 |	6 |? | ? |
| SEIR56SEIR57 | 116.5 | 117.1 | -49.4 | -49.2	|4 |? | ? |

I've provided code to produce a plot with the values, once you have entered them here:

In [ ]:
axial_depths = [700.0, ]
Na8 = [1.9, ]


fig, ax = plt.subplots(dpi=100)

ax.scatter(axial_depths, Na8, c='C2', s=100)

ax.set_xlabel('Axial depth (m)')
ax.set_ylabel('Na$_8$')

plt.show()

## 4. Modelling the global array

It has been suggested that the global array you have just obtained a version of is controlled by variations in mantle temperature. By constructing a simple model, we can test whether this is a feasible explanation.

We will add a line to the plot above where each point on the line represents a different value of $T_p$.

The model will need to take mantle potential temperature $T_p$ as an input, and estimate the Na>sub>2</sub>O content of the primary melt and the axial depth of the ridge. In the lectures we have covered the equations needed to do this. The first step is to calculate the melt fraction generated for a given $T_p$ at a spreading centre. There are many ways of doing this (including reading numbers off the figures from the lecture!), but there is a convenient python package that can help us:

In [ ]:
import pyMelt as m

The way the [pyMelt](https://pymelt.readthedocs.io/en/latest/) package is structured requires us to define a mantle lithology and build a mantle object from it:

In [ ]:
# Only need to run this once!
lz = m.lithologies.matthews.klb1()
mantle = m.mantle([lz], [1.0], ['lz'])

This tells pyMelt to use a set of equations that paramterise KLB1 lherzolite melting published by [Matthews et al. (2021)](https://doi.org/10.1029/2020GC009157). Then we can perform an adiabatic melting calculation at a given $T_p$ (in ˚C), and produce a plot to visualise the results:

In [ ]:
# This line does the calculation:
column = mantle.adiabaticMelt(1360.0, Pstart=4.0)

# This line makes a nice plot
f, a = column.plot()

### Q4.1: Find in your lecture notes the equation for obtaining crustal thickness from the information plotted above. Can you make a rough estimate of what it should be?

*Your answer here...*

In [ ]:
# You might want to write some python code to help!

We can have pyMelt do the calculation too, but first we must tell it to creating a spreading centre:

In [ ]:
mor = m.geosettings.spreadingCentre(column)
ref_tc = mor.tc
ref_tc

### Q4.2: Does the calculated value differ from your own answer? Why?

*Your answer here...*

We expect that mid-ocean ridges are close to isostatic equilibrium, so variations in axial depth should be driven by density differences in the rock beneath them. There will be two main components to this: i) variations in crustal thickness and ii) variations in mantle temperature. To make the calculations easier we will ignore the contribution of mantle temperature and focus on crustal thickness.

### Q4.3: Derive an expression linking a change in ridge bathymetry to a change in crustal thickness.
*Assume uniform densities (values below) for the crust, ocean, and mantle.*
| Material | Density (kg m<sup>-3</sup>) |
|---|---|
| Water | 1000 |
| Crust | 3000 |
| Convecting Mantle | 3300 |

*Your answer here...*

Using pyMelt we can calculate the thickness of oceanic crust as a function of mantle $T_p$, and therefore the change in bathymetry from a reference value. We will use a $T_p$ of 1360˚C as the reference case (for which we calculated the crustal thickness above).

### Q4.4: By modifying the code below, create a loop that calculates the crustal thickness for a range of $T_p$:

In [ ]:
# Create an empty list to store the answers in
tcs = []

# This is for Q4.6:
Fmax = []

for Tp in np.linspace(1300, 1550, 6): # Run the code below for 6 values of Tp spaced evenly between 1300 and 1550. Increase this is you want!
    
    result = 0.0 # Change this bit!
    

    tcs.append(result)
    
    # Code for Q4.6 can go here:


print(tcs)

We can convert this to a numpy array to make calculations easier:

In [ ]:
tcs = np.array(tcs)
Fmax = np.array(Fmax)

### Q4.5: Now use the expression you derived above to calculate the bathymetry relative to the $T_p$ = 1360˚C case:

In [ ]:
# Your code here...


We must now calculate the primary Na<sub>2</sub>O content of the magmas associated with each $T_p$. 

If we use a highly simplified melting model, the Na<sub>2</sub>O content of the mantle melts can be calculated from the following expression for accumulated fractional melts which was developed by [Plank et al., 1995](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/95JB01148), a paper which is worth a look.

$$
C_L = \frac{2C_0}{X_{max}} \left[ 1 + \frac{D \left[ (1 - X_{max})^{((1/D) - 1))} -1 \right]}{X_{max}(D+1)} \right]
$$

In this case, $C_L$ is the liquid composition, $C_0$ is the mantle source composition (you can try 0.3 wt% for Na<sub>2</sub>O and $D$, the partition coefficient during melting (try a constant of 0.01 for Na).

You can extract the melt fraction at the top of the melting column (i.e., the maximum melt fraction) from the pyMelt results using:

In [ ]:
mor.F.max()

### Q4.6: Add Fmax into the loop above so you extract Fmax and crustal thickness at the same time. Check Fmax makes sense.

In [ ]:
# Print Fmax here.

### Q4.7: Use these values of Fmax to calculate Na<sub>2</sub>O of the magma produced for each $T_p$:

In [ ]:
# Your code here

### Q4.8: Create a version of the global array plot with the model superimposed.

*Remember that the bathymetrys calculated are **relative** bathymetrys. You will need to add an offset to it to match the data.

In [ ]:
fig, ax = plt.subplots(dpi=100)

ax.scatter(axial_depths, Na8, c='C2', s=100)

ax.set_xlabel('Axial depth (m)')
ax.set_ylabel('Na$_8$')


ax.plot(relative_bathymetry*1000 + 3500, Na2O)

plt.show()

### Q4.9: What range of $T_p$ is required to explain the global array? What might affect your answer?

*Your answer here...*